## Notebook purpose:
In this notebook we link patient level data (amyloid diagnosis labbels, echos data, demographics, icd codes, clinical notes) to patients in our cohort. We then perform cohort analytics.

## Datasets:

* **Amyloidosis Patients Cohort Entry Labels** is the list of patients in our cohort with labels. We created labels from 2 sources, **cohort patient labels** and **Amyloidosis Patients Cohort Entry**. We want to link patient level data to each patient in the cohort.
   * **cohort patient labels** are labels obtained from 3 sources:
     * Labels from Mayo Reports, Cardiac Path Reports, Pyp Reports
     * Labels from clinical chart review for TTR
     * Labels from clinical chart review for AL
  * **Amyloidosis Patients Cohort Entry** is the list of patients in our cohort. We want to link patient level data to each patient in the cohort. Includes a column that indicates if a patient has been prescribed **Tafamidis**. This drug is prescribed for TTR Amyloidosis (hTTR and wTTR).
 
* **Amyloidosis Patients EchoMaster** links patients to echo data. We care about organizing echos for a patient with respect to a diagnosis date.

* **Amyloidosis Patients HF_Subtype** links patients to Amyloid ICD codes. We care about the column indicating if a patient has an Amyloid ICD code - SILVER STANDARD for amyloid diagnosis

* **Amyloidosis Patients Demographics** links patients to demographics data. We care about the column indicating a patient's age at cohort entry.

* **Amyloidosis Patients OutpatientNotesDeid** links patients to clinical notes data.

### Output
**Amyloidosis Patients Cohort Entry - Labeled** is the dataframe for patients in our cohort.

The columns are described below:
*ICD Codes*
* `amyloidosis_ICD_code`: 1 if patient has an amyloidosis ICD code, else 0.

*Demographics*
* `Age_cohort`: patient age at cohort entry

*Label Columns*
* `label__amyloid_diagnosis`: Confirmed diagnosis for amyloid. Values: POSITIVE, NEGATIVE, INDETERMINATE, CHART_REVIEW.
* `label__amyloid_subtype_diagnosis`: Confirmed diagnosis for amyloid subtype. Values: TTR, AL, INDETERMINATE, CHART_REVIEW, NaN.
* `label__ttr_amyloid_subtype_diagnosis`: Confirmed diagnosis for TTR amyloid subtype. Values: HTTR, INDETERMINATE (defaults to WTTR), CHART_REVIEW, NaN.
* `label__amyloid_diagnosis_date`: Date of confirmed diagnosis.
* `full_chart_review`: 1 if we have the latest chart review information.
* `label__definitive`: 1 for `label__amyloid_diagnosis` with value POSITIVE or NEGATIVE, else 0.
* `label__missing_diagnosis`: 1 if `label__definitive` with value 0 and `amyloidosis_ICD_code` with value 1 and `full_chart_review` with value 0, else 0.

*Echos*
* `echos_cohort_entry`: 1 if patient has echos of interest in EchoMaster.csv, else 0.

*Notes*
* `notes_cohort_entry`: 1 if patient has clinical notes in Amyloidosis Patients OutpatientNotesDeid.csv, else 0.

*Amyloid Cases*
* `patient_group__amyloid_cases`: 1 if patient `label__amyloid_diagnosis` value is POSITIVE or `amyloidosis_ICD_code`  value is 1 and `label__amyloid_diagnosis` value is not NEGATIVE.

*Controls*
* `patient_group__HF_control`: 1 if `HF_cohort_entry` value is 1 and `patient_group__amyloid_cases` is not 1
* `patient_group__non_HF_control`: 1 if `HF_cohort_entry` value is NaN and `patient_group__amyloid_cases` is not 1


In [1]:
import pandas as pd
from pathlib import Path

from file_parsing.cohort_file import load_cohort_entry
from file_parsing.echomaster_file import load_echomaster
from file_parsing.hf_subtype_file import load_hf_subtype
from file_parsing.demographics_file import load_demographics

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [2]:
# The path to the Amyloid data
DATASET_PATH = Path("/data/datasets/Amyloidosis/")

# File with echos data
echo_master_path = DATASET_PATH / "Amyloidosis Patients EchoMaster.csv"

# File with patient ICD codes - we care about the column indicating if a patient has an Amyloid ICD code - SILVER STANDARD for amyloid diagnosis
hf_subtype_path = DATASET_PATH / "Amyloidosis Patients HF_Subtype.csv"

# File with patient demographics - we care about the column indicating patient age at cohort entry
demographics_path = DATASET_PATH / "Amyloidosis Patients Demographics.csv"

# The cohort file to which we add diagnosis and demographic info
cohort_entry_path = DATASET_PATH / "Amyloidosis Patients Cohort Entry.csv"

# clinical notes
notes_path = DATASET_PATH / "Amyloidosis Patients OutpatientNotesDeid.csv"

""" 
Cohort amyloid labels before chart review
amyloid_diagnosis_labels_path = (
    DATASET_PATH
    / "patient_amyloid_diagnosis"
    / "cohort_amyloid_labels.csv"
)
"""

# Cohort amyloid labels after chart review
chart_reviewed_cohort_amyloid_labels_path = (
    DATASET_PATH / "2023 pull" / "patient_amyloid_diagnosis" / "cohort_amyloid_labels__chart_reviewed_2023.csv"# "cohort_amyloid_labels__chart_reviewed.csv"
)


### Load Patient Level Labels

In [3]:
# Read the patient level diagnosis data
labels = pd.read_csv(chart_reviewed_cohort_amyloid_labels_path)

print(labels.columns)
# we keep these columns but you can also include columns specific to the datasets:
# cardiac path reports, pyp reports, and mayo labs
labels = labels[
    [
        "ir_id",
        "label__amyloid_diagnosis_date",
        "label__amyloid_diagnosis",
        "label__amyloid_subtype_diagnosis",
        "label__ttr_amyloid_subtype_diagnosis",
        "full_chart_review", # Flag for full chart review
        "label__chart_review", # Flag for chart review
        "pyp_or_tafamidis_only", # Flag for pyp or tafamidis
        
    ]
]
labels["label__amyloid_diagnosis_date"] = pd.to_datetime(
    labels["label__amyloid_diagnosis_date"]
)

# TODO: Add condition to check if full chart review was done
labels["label__definitive"] = labels["label__amyloid_diagnosis"].isin(["POSITIVE", "NEGATIVE"]).astype(int)


Index(['ir_id', 'label__amyloid_diagnosis', 'label__amyloid_subtype_diagnosis',
       'label__ttr_amyloid_subtype_diagnosis', 'label__amyloid_diagnosis_date',
       'full_chart_review', 'pyp_or_tafamidis_only', 'label__chart_review',
       'diagnosis__chart_review', 'cardiac_path__amyloid_diagnosis',
       'pyp__amyloid_diagnosis', 'mayo__amyloid_diagnosis',
       'mayo__amyloid_subtype_diagnosis',
       'mayo__ttr_amyloid_subtype_diagnosis', 'final__amyloid_diagnosis',
       'final__amyloid_diagnosis_date', 'final__amyloid_subtype_diagnosis',
       'final__ttr_amyloid_subtype_diagnosis', 'chart_reviews__Amyloid_type',
       'chart_reviews__Method_of_diagnosis',
       'chart_reviews__amyloid_diagnosis_date',
       'chart_reviews__Age_at_Diagnosis', 'chart_reviews__amyloid_diagnosis',
       'chart_reviews__amyloid_subtype_diagnosis',
       'chart_reviews__ttr_amyloid_subtype_diagnosis',
       'merge_chart_reviews_consistency',
       'merge_chart_reviews_consistency_descri

In [ ]:
labels.label__chart_review.value_counts()

In [ ]:
print(
    f'We have {len(labels["ir_id"].unique())} patients with a gold standard amyloid diagnosis.'
)

print(labels["label__amyloid_diagnosis"].value_counts(dropna=False))

print(labels["label__definitive"].value_counts(dropna=False))

print(
    labels[labels["label__amyloid_diagnosis"] == "POSITIVE"][
        "label__amyloid_subtype_diagnosis"
    ].value_counts(dropna=False)
)

print(
    labels[labels["label__amyloid_subtype_diagnosis"] == "TTR"][
        "label__ttr_amyloid_subtype_diagnosis"
    ].value_counts(dropna=False)
)

### Load Cohort Entry (Includes Tafimidis)

In [6]:
# read file and drop the last 2 rows, containing information from SQL operation
cohort_entry = load_cohort_entry()
assert len(cohort_entry.ir_id.unique()) == cohort_entry.shape[0], "cohort entry has more than one record per patient"



In [ ]:
print(
    f'We have {len(cohort_entry[cohort_entry["Tafamidis_cohort_entry"]==1]["ir_id"].unique())} patients with Amyloid because they take Tafimifis medication.'
)
print(
    f'We have {len(cohort_entry["ir_id"].unique())} patients in the cohort.'
)

### Load HF Subtype (ICD Codes)

In [8]:
# read file and drop the last 2 rows, containing information from SQL operation
hf_subtype = load_hf_subtype()

In [ ]:
print(
    f'We have {len(hf_subtype[hf_subtype["Amyloidosis"] == 1]["ir_id"].unique())} patients with an Amyloid ICD code.'
)



### Load Demographics Data

In [10]:
# read file and drop the last 2 rows, containing information from SQL operation
demographics = load_demographics()

### Create labeled cohort file

In [11]:
cohort = cohort_entry.merge(labels, on="ir_id", how="left")
cohort = cohort.merge(hf_subtype, on="ir_id", how="left")
cohort = cohort.merge(demographics, on="ir_id", how="left")

# These columns have value 1 if True, 0 if False, or NaN (due to the merge)
# Filling NaN with 0 will help with operations between these columns
cohort["label__definitive"].fillna(0, inplace=True)
cohort["label__definitive"]= cohort["label__definitive"].astype(int)
cohort["label__chart_review"].fillna(0, inplace=True)
cohort["label__chart_review"]= cohort["label__chart_review"].astype(int)
cohort["pyp_or_tafamidis_only"].fillna(0, inplace=True)
cohort["pyp_or_tafamidis_only"]= cohort["pyp_or_tafamidis_only"].astype(int)

cohort["Tafamidis_cohort_entry"].fillna(0, inplace=True)
cohort["Tafamidis_cohort_entry"]= cohort["Tafamidis_cohort_entry"].astype(int)

cohort["Amyloidosis"].fillna(0, inplace=True)
cohort["Amyloidosis"]= cohort["Amyloidosis"].astype(int)

#TODO: Add new columns from the full chart review
cohort["full_chart_review"].fillna(0, inplace=True)
cohort["full_chart_review"]= cohort["full_chart_review"].astype(int)

In [12]:
cohort.columns

Index(['ir_id', 'HF_cohort_entry', 'HF_cohort_entry_date', 'CA_cohort_entry',
       'CA_cohort_entry_date', 'CM_cohort_entry', 'CM_cohort_entry_date',
       'PYP_cohort_entry', 'PYP_cohort_entry_date', 'Tafamidis_cohort_entry',
       'Tafamidis_cohort_entry_date', 'cMRI_cohort_entry',
       'cMRI_cohort_entry_date', 'HF_stricter_definition_date',
       'label__amyloid_diagnosis_date', 'label__amyloid_diagnosis',
       'label__amyloid_subtype_diagnosis',
       'label__ttr_amyloid_subtype_diagnosis', 'full_chart_review',
       'label__chart_review', 'pyp_or_tafamidis_only', 'label__definitive',
       'HFdx', 'HFdx_date', 'Age_HFdx_date', 'HF_Dx_setting', 'HFrEF_anytime',
       'HFrEF_anytime_echodate', 'HFrEF', 'HFrEF_initialechodate', 'HFmEF',
       'HFmEF_initialechodate', 'HFrecEF', 'HFrecEF_initialechodate',
       'HFrecEF_followupecho', 'HFrecEF_daystorec', 'HFpEF',
       'HFpEF_initialechodate', 'Ischemic_cardiomyopathy',
       'ischemic_cardiomyopathy_date', 'Non_isc

In [13]:
cohort["label__missing_diagnosis"] = (
    ~cohort["label__definitive"]
    & cohort["Amyloidosis"]
    & ~cohort["full_chart_review"]
)



In [ ]:
cohort["label__missing_diagnosis"].value_counts(dropna=False)

### Load Echos Data

In [15]:
echos = load_echomaster()

# filter the echos to keep studies that are not limited and that have desired desired type 
keep_echo_types = ["Transthoracic", "Exercise Stress", "Pharmacological Stress", "Stress Type Unknown"] 
echos = echos.loc[(echos["echo_type"].isin(keep_echo_types)) & (echos["limited_echo"] == 0)]
# drop records with an echo extractor id
echos = echos[echos["echo_extractor_id"].isna()]
# sort chronologically so that the aggregation gives us a list of echos information in chronological order
echos.sort_values(by=['echo_date'], inplace=True)

# Group Rows into List on All columns
echos_by_patient = echos.groupby("ir_id").agg(list)
echos_by_patient.reset_index(inplace=True)

In [ ]:
print(
    f'We have {len(echos)} echos.'
)
print(
    f'We have {len(echos_by_patient["ir_id"].unique())} patients with echos.'
)

### Add Echos Flag to Cohort

In [17]:
ir_id_with_echo = pd.DataFrame(
    {
        "ir_id": echos.ir_id.unique(),
        "echos_cohort_entry": 1
    }
)
cohort = cohort.merge(ir_id_with_echo, on="ir_id", how="left")
cohort["echos_cohort_entry"].fillna(0, inplace=True)
cohort["echos_cohort_entry"] = cohort["echos_cohort_entry"].astype(int)

In [ ]:
cohort["echos_cohort_entry"].value_counts(dropna=False)

### Load Notes Data

In [19]:
notes = pd.read_csv(notes_path)
notes["created_date_key"] = pd.to_datetime(notes["created_date_key"])
# sort chronologically so that the aggregation gives us a list of notes information in chronological order
notes.sort_values(by=["ir_id", "created_date_key"], inplace=True)


### Add Notes Flag to Cohort

In [20]:
ir_id_with_notes = pd.DataFrame(
    {
        "ir_id": notes.ir_id.unique(),
        "notes_cohort_entry": 1
    }
)
cohort = cohort.merge(ir_id_with_notes, on="ir_id", how="left")
cohort["notes_cohort_entry"].fillna(0, inplace=True)
cohort["notes_cohort_entry"] = cohort["notes_cohort_entry"].astype(int)

In [ ]:
cohort["notes_cohort_entry"].value_counts(dropna=False)

### Save Cohort File

In [22]:
# The cohort file to which we add diagnosis and demographic info
labeled_cohort_path = DATASET_PATH / "2023 pull" / "Amyloidosis Patients Cohort Entry.csv"

In [ ]:
cohort["label__amyloid_diagnosis"].value_counts(dropna=False)

In [ ]:
cohort[cohort["label__amyloid_diagnosis"]=="POSITIVE"]["label__amyloid_subtype_diagnosis"].value_counts(dropna=False)

In [ ]:
cohort[cohort["label__amyloid_subtype_diagnosis"]=="TTR"]["label__ttr_amyloid_subtype_diagnosis"].value_counts(dropna=False)

In [ ]:
cohort[cohort["echos_cohort_entry"]==1]["label__amyloid_diagnosis"].value_counts(dropna=False)

In [ ]:
cohort[cohort["notes_cohort_entry"]==1]["label__amyloid_diagnosis"].value_counts(dropna=False)

In [ ]:
# ICD code
cohort["Amyloidosis"].value_counts(dropna=False)

In [ ]:
cohort["pyp_or_tafamidis_only"].value_counts(dropna=False)

In [29]:
cohort["patient_group__amyloid_cases"] = (cohort["label__amyloid_diagnosis"] == "POSITIVE") | (
    (cohort["Amyloidosis"] == 1) & (cohort["label__amyloid_diagnosis"] != "NEGATIVE")
)


In [ ]:
cohort["patient_group__amyloid_cases"].value_counts(dropna=False)


In [31]:
cohort["patient_group__HF_control"] = (~cohort["patient_group__amyloid_cases"]) & (cohort["HF_cohort_entry"].notna())
cohort["patient_group__non_HF_control"] = (~cohort["patient_group__amyloid_cases"]) & (cohort["HF_cohort_entry"].isna())


In [ ]:
cohort["patient_group__HF_control"].value_counts(dropna=False)

In [ ]:
cohort["patient_group__non_HF_control"].value_counts(dropna=False)

In [34]:
cohort.to_csv(labeled_cohort_path, index=False)

In [35]:
cohort.columns

Index(['ir_id', 'HF_cohort_entry', 'HF_cohort_entry_date', 'CA_cohort_entry',
       'CA_cohort_entry_date', 'CM_cohort_entry', 'CM_cohort_entry_date',
       'PYP_cohort_entry', 'PYP_cohort_entry_date', 'Tafamidis_cohort_entry',
       'Tafamidis_cohort_entry_date', 'cMRI_cohort_entry',
       'cMRI_cohort_entry_date', 'HF_stricter_definition_date',
       'label__amyloid_diagnosis_date', 'label__amyloid_diagnosis',
       'label__amyloid_subtype_diagnosis',
       'label__ttr_amyloid_subtype_diagnosis', 'full_chart_review',
       'label__chart_review', 'pyp_or_tafamidis_only', 'label__definitive',
       'HFdx', 'HFdx_date', 'Age_HFdx_date', 'HF_Dx_setting', 'HFrEF_anytime',
       'HFrEF_anytime_echodate', 'HFrEF', 'HFrEF_initialechodate', 'HFmEF',
       'HFmEF_initialechodate', 'HFrecEF', 'HFrecEF_initialechodate',
       'HFrecEF_followupecho', 'HFrecEF_daystorec', 'HFpEF',
       'HFpEF_initialechodate', 'Ischemic_cardiomyopathy',
       'ischemic_cardiomyopathy_date', 'Non_isc